In [17]:
import napari
import numpy as np
import imageio

# Load the volume
#volume = np.load("../data/tesi_mat/exp_2/case_2/qualitative/21.npy")  # Replace with your actual file
volume = np.load("../data/id_volumes/id_38.npy")

Swap axes only for R2Gaus, and not FBP

In [18]:
# Swap axes
volume = np.swapaxes(volume, 0, 1)
volume = np.swapaxes(volume, 0, 2) # shape: (256, 256, 256)

# Invert projections
volume = volume[::-1, :, :]

# Rotate counterclockwise 90 degrees
volume = np.rot90(volume, k=1, axes=(1, 2))

# Flip along vertical axis
volume = volume[:, :, ::-1]

In [19]:
# crop central portion
center = [dim // 2 for dim in volume.shape] # center coordinates
radius = 128
angle = np.pi/4
margin = 10

side = 2 * radius * np.cos(angle)

offset_depth = 128
offset_height = int(side / 2)
offset_width = int(side / 2)

cropped_volume = volume[
    center[0] - offset_depth: center[0] + offset_depth,
    center[1] - offset_height + margin: center[1] + offset_height - margin,
    center[2] - offset_width + margin: center[2] + offset_width - margin,
]
cropped_volume.shape

(256, 160, 160)

In [20]:
cropped_volume = cropped_volume[::-1, :, :]
cropped_volume.shape

(256, 160, 160)

In [21]:
# enhance intensities
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from skimage import exposure

def enhance_volume(volume, percetile_left, percentile_right, gamma):
    """
    Enhance the blobs in a 3D volume by normalizing, applying contrast stretching, 
    and using gamma correction.
    
    Parameters:
    - volume: numpy array of shape (D, H, W)
    - gamma: float, gamma correction factor (>1 to emphasize higher intensities)
    
    Returns:
    - enhanced_volume: numpy array with values between 0 and 1
    """
    # Step 1: Min-Max Normalization
    volume_normalized = (volume - np.min(volume)) / (np.max(volume) - np.min(volume))

    # Step 2: Contrast Stretching
    p1, p2 = np.percentile(volume_normalized, (percetile_left, percentile_right))
    volume_contrast = exposure.rescale_intensity(volume_normalized, in_range=(p1, p2))
    
    # Step 3: Gamma Correction
    volume_gamma = exposure.adjust_gamma(volume_contrast, gamma)
    
    return volume_gamma

def adjust_gamma(volume, gamma):
    """
    Normalize volume beteen 0 and 1, and apply gamma correction.
    """
    volume_normalized = (volume - np.min(volume)) / (np.max(volume) - np.min(volume))
    volume_gamma = exposure.adjust_gamma(volume_normalized, gamma)
    return volume_gamma

def plot_histograms(volume_list, bins=50, labels=None, colors=None, alpha=0.5, log=True):
    if labels is None:
        labels = [f'Volume {i+1}' for i in range(len(volume_list))]
    if colors is None:
        colors = plt.cm.tab10.colors
    plt.figure(figsize=(8, 4))
    for i, arr in enumerate(volume_list):
        color = colors[i % len(colors)]
        plt.hist(arr.flatten(), bins=bins, label=labels[i], color=color, alpha=alpha, edgecolor='black', log=log)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histograms of 3D Volumes')
    plt.legend()
    plt.grid(True)
    plt.show()

def normalize_to_01(volume):
    """Normalize a 3D NumPy array to the range [0, 1]."""
    v_min = np.min(volume)
    v_max = np.max(volume)
    return (volume - v_min) / (v_max - v_min)

def clip_and_normalize(volume, min_percentile=1, max_percentile=99):
    """Clip outliers based on percentiles and normalize to [0, 1]."""
    lower_bound = np.percentile(volume, min_percentile)
    upper_bound = np.percentile(volume, max_percentile)
    clipped_volume = np.clip(volume, lower_bound, upper_bound)
    return normalize_to_01(clipped_volume)


cropped_volume_norm = normalize_to_01(cropped_volume)
cropped_volume_norm_enh = enhance_volume(cropped_volume_norm, percetile_left=0, percentile_right=100, gamma=1.5)

In [22]:
# Start Napari viewer
viewer = napari.Viewer(ndisplay=3)

# Add volume with "twilight_shifted" colormap
layer = viewer.add_image(cropped_volume_norm_enh, colormap="twilight_shifted", rendering='mip')

# Adjust the camera angle and location
viewer.camera.angles = (45, 70, 45)  # Adjust these values (yaw, pitch, roll)
viewer.camera.center = (128, 80, 80)  # Adjust for desired location
viewer.camera.zoom = 2.8  # Zoom out (lower values mean more zoomed out)

# Take a snapshot
screenshot = viewer.screenshot(canvas_only=True)  # Get the screenshot as a NumPy array
imageio.imwrite("../data/snap/snapshot.png", screenshot)  # Save as an image file

#napari.run()  # Keep the viewer open
viewer.close()